# Description

This notebook is a template notebook that is intended to be run across different parameters.

Based on the settings below, it loads an input file with paragraph pairs (original and revised) and uses the LLM-as-a-Judge approach to evaluate the quality of the paragraphs.

# Modules

In [1]:
import pandas as pd
from IPython.display import display
from langchain.cache import SQLiteCache
from langchain.globals import set_llm_cache
from proj import conf
from proj.utils import llm_pairwise

# Settings/paths

In [2]:
# Input manuscript
REPO = None

INPUT_FILE = None
OUTPUT_FILE = None

# Model and its parameters
LLM_JUDGE = None
TEMPERATURE = None
MAX_TOKENS = 2000
SEED_INIT = 0

# Evaluation parameters
N_REPS = None
THROW_IF_FAILED = False

In [3]:
# Parameters
REPO = "pivlab/manubot-ai-editor-code-test-ccc-manuscript"
INPUT_FILE = "/home/miltondp/projects/others/manubot/manubot-ai-editor-code/base/results/paragraph_match/ccc-manuscript--gpt-3.5-turbo.pkl"
OUTPUT_FILE = "/home/miltondp/projects/others/manubot/manubot-ai-editor-code/base/results/llm_pairwise/ccc-manuscript--gpt-3.5-turbo--openai_gpt-3.5-turbo.pkl"
LLM_JUDGE = "openai:gpt-3.5-turbo"
TEMPERATURE = 0.5
MAX_TOKENS = 2000
SEED_INIT = 0
N_REPS = 5


In [4]:
conf.common.LLM_CACHE_DIR.mkdir(parents=True, exist_ok=True)
display(conf.common.LLM_CACHE_DIR)

PosixPath('/home/miltondp/projects/others/manubot/manubot-ai-editor-code/base/results/llm_cache')

# Set default LangChain cache file

In [5]:
default_cache_file = conf.common.LLM_CACHE_DIR / "default.db"
display(default_cache_file)
set_llm_cache(SQLiteCache(database_path=str(default_cache_file)))

PosixPath('/home/miltondp/projects/others/manubot/manubot-ai-editor-code/base/results/llm_cache/default.db')

# Load paragraphs

In [6]:
df = pd.read_pickle(INPUT_FILE)

In [7]:
df.shape

(31, 3)

In [8]:
df.head()

,section,original,modified
0,abstract,Correlation coefficients are widely used to id...,"In transcriptomics, identifying patterns in ge..."
1,introduction,New technologies have vastly improved data col...,Recent advancements in data collection have le...
2,introduction,"In transcriptomics, many analyses start with e...","In the field of transcriptomics, many analyses..."
3,introduction,The Pearson and Spearman correlation coefficie...,The Pearson and Spearman correlation coefficie...
4,results,The CCC provides a similarity measure between ...,The CCC calculates the similarity between pair...


In [9]:
df.iloc[0]["original"]

'Correlation coefficients are widely used to identify patterns in data that may be of particular interest. In transcriptomics, genes with correlated expression often share functions or are part of disease-relevant biological processes. Here we introduce the Clustermatch Correlation Coefficient (CCC), an efficient, easy-to-use and not-only-linear coefficient based on machine learning models. CCC reveals biologically meaningful linear and nonlinear patterns missed by standard, linear-only correlation coefficients. CCC captures general patterns in data by comparing clustering solutions while being much faster than state-of-the-art coefficients such as the Maximal Information Coefficient. When applied to human gene expression data, CCC identifies robust linear relationships while detecting nonlinear patterns associated, for example, with sex differences that are not captured by linear-only coefficients. Gene pairs highly ranked by CCC were enriched for interactions in integrated networks b

In [10]:
df.iloc[0]["modified"]

'In transcriptomics, identifying patterns in gene expression data is crucial for understanding biological processes. The traditional linear correlation coefficients may not capture all the complexities in gene relationships. To address this, we propose the Clustermatch Correlation Coefficient (CCC), a machine learning-based coefficient that can reveal both linear and nonlinear patterns efficiently. Compared to standard coefficients like the Maximal Information Coefficient, CCC is faster and can detect biologically meaningful patterns that are missed by linear-only methods. When applied to human gene expression data, CCC not only identifies robust linear relationships but also uncovers nonlinear patterns, such as sex differences. Additionally, CCC can detect functional relationships between genes that are overlooked by linear-only methods, as shown by enrichment in integrated networks. Overall, CCC is a versatile and efficient tool that can be applied to genome-scale data and various da

# Test run

In [11]:
t_json = llm_pairwise(
    df.iloc[0]["original"],
    df.iloc[0]["modified"],
    df.iloc[0]["section"],
    model_name=LLM_JUDGE,
    model_params={
        "temperature": TEMPERATURE,
        "max_tokens": MAX_TOKENS,
        "model_kwargs": {
            "seed": SEED_INIT,
        },
    },
    verbose=True,
)



> Entering new LLMChain chain...
Prompt after formatting:
System: You are an expert copyeditor with ample experience in scientific writing. You are assessing the quality of two versions of a paragraph from the Abstract of a scientific article.
Human: Evaluate the quality of the following paragraph by writing a list with positive (if any) and/or negative (if any) aspects on the following areas: 1) has a clear sentence structure, 2) is easy to follow, 3) is correct in grammar, 4) has no spelling errors.

Paragraph A: Correlation coefficients are widely used to identify patterns in data that may be of particular interest. In transcriptomics, genes with correlated expression often share functions or are part of disease-relevant biological processes. Here we introduce the Clustermatch Correlation Coefficient (CCC), an efficient, easy-to-use and not-only-linear coefficient based on machine learning models. CCC reveals biologically meaningful linear and nonlinear patterns missed by standard


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: You are an expert copyeditor with ample experience in scientific writing. You are assessing the quality of two versions of a paragraph from the Abstract of a scientific article.
Human: Evaluate the quality of the following paragraph by writing a list with positive (if any) and/or negative (if any) aspects on the following areas: 1) has a clear sentence structure, 2) is easy to follow, 3) is correct in grammar, 4) has no spelling errors.

Paragraph A: Correlation coefficients are widely used to identify patterns in data that may be of particular interest. In transcriptomics, genes with correlated expression often share functions or are part of disease-relevant biological processes. Here we introduce the Clustermatch Correlation Coefficient (CCC), an efficient, easy-to-use and not-only-linear coefficient based on machine learning models. CCC reveals biologically meaningful linear and nonlinear patterns


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
System: You are an expert copyeditor with ample experience in scientific writing. You are assessing the quality of two versions of a paragraph from the Abstract of a scientific article.
Human: Evaluate the quality of the following paragraph by writing a list with positive (if any) and/or negative (if any) aspects on the following areas: 1) has a clear sentence structure, 2) is easy to follow, 3) is correct in grammar, 4) has no spelling errors.

Paragraph A: Correlation coefficients are widely used to identify patterns in data that may be of particular interest. In transcriptomics, genes with correlated expression often share functions or are part of disease-relevant biological processes. Here we introduce the Clustermatch Correlation Coefficient (CCC), an efficient, easy-to-use and not-only-linear coefficient based on machine learning models. CCC reveals biologically meaningful linear and nonlinear patterns


> Finished chain.


In [12]:
t_json

{'best': 'Paragraph A',
 'rationale': 'Paragraph A has a slightly more sophisticated language use and introduces the concept of Clustermatch Correlation Coefficient (CCC) more explicitly, providing a comprehensive overview of its applications and advantages. Both paragraphs are well-structured, easy to follow, grammatically correct, and free of spelling errors, but Paragraph A stands out for its detailed explanation and the depth of information provided.'}

In [13]:
type(t_json)

dict

# Run

Since models are stochastic, we run the pairwise comparison many times.

Here I use a cache to avoid hitting an external API multiple times.

In [14]:
results = []

In [15]:
for rep_idx in range(N_REPS):
    # we cache prompt/results by repetition
    output_cache_file = conf.common.LLM_CACHE_DIR / f"rep{rep_idx}.db"
    set_llm_cache(SQLiteCache(database_path=str(output_cache_file)))

    print(f"{str(rep_idx).zfill(2)} ({output_cache_file.name}): ", end="", flush=True)

    for par_idx, par in df.iterrows():
        print(".", end="", flush=True)

        res = llm_pairwise(
            par["original"],
            par["modified"],
            par["section"],
            model_name=LLM_JUDGE,
            model_params={
                "temperature": TEMPERATURE,
                "max_tokens": MAX_TOKENS,
                "model_kwargs": {
                    "seed": SEED_INIT + rep_idx,
                },
            },
            throw_if_failed=THROW_IF_FAILED,
            verbose=False,
        )

        results.append(
            {
                "rep_index": rep_idx,
                "paragraph_index": par_idx,
                "paragraph_section": par["section"],
                "winner": res["best"],
                "rationale": res["rationale"],
            }
        )

    print(flush=True)

00 (rep0.db): 

.

.

.

.

...

.

.

.

.

.

# Process results

In [16]:
winner_matchings = {
    "Paragraph A": "-1",  # Original
    "Paragraph 1": "1",  # Modified
    "tie": "0",
}

In [17]:
df_results = pd.DataFrame(results)

In [18]:
df_results.shape

(155, 5)

In [19]:
df_results.head()

,rep_index,paragraph_index,paragraph_section,winner,rationale
0,0,0,abstract,tie,Both paragraphs exhibit clear sentence structu...
1,0,1,introduction,tie,Both paragraphs exhibit clear sentence structu...
2,0,2,introduction,Paragraph 1,Both paragraphs exhibit clear sentence structu...
3,0,3,introduction,Paragraph 1,Paragraph 1 is slightly better as it contains ...
4,0,4,results,Paragraph A,Paragraph A has a slightly clearer sentence st...


In [20]:
df_results["winner"].value_counts()

winner
Paragraph 1    80
tie            64
Paragraph A    11
Name: count, dtype: int64

In [21]:
df_results = df_results[df_results["winner"].isin(winner_matchings.keys())]

In [22]:
df_results.shape

(155, 5)

In [23]:
df_results = df_results.assign(
    winner_score=df_results["winner"].replace(winner_matchings).apply(float)
)

In [24]:
df_results.shape

(155, 6)

In [25]:
df_results.head()

,rep_index,paragraph_index,paragraph_section,winner,rationale,winner_score
0,0,0,abstract,tie,Both paragraphs exhibit clear sentence structu...,0.0
1,0,1,introduction,tie,Both paragraphs exhibit clear sentence structu...,0.0
2,0,2,introduction,Paragraph 1,Both paragraphs exhibit clear sentence structu...,1.0
3,0,3,introduction,Paragraph 1,Paragraph 1 is slightly better as it contains ...,1.0
4,0,4,results,Paragraph A,Paragraph A has a slightly clearer sentence st...,-1.0


In [26]:
df_results.dtypes

rep_index              int64
paragraph_index        int64
paragraph_section     object
winner                object
rationale             object
winner_score         float64
dtype: object

In [27]:
df_results.groupby("paragraph_section")["winner_score"].mean()

paragraph_section
abstract                  0.400000
discussion                0.600000
introduction              0.666667
methods                   0.175000
results                   0.520000
supplementary material    0.400000
Name: winner_score, dtype: float64

# Save

In [28]:
df_results.to_pickle(OUTPUT_FILE)